In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numba import njit

1D (VMC)

In [ ]:
def trial_wf(alpha, z):
    return np.exp(-alpha*(z**2))

def prob_density(alpha, z):
    return trial_wf(alpha,z)**2/np.sqrt(np.pi/alpha)

def E_local(alpha, z):
    return alpha + (z**2)*(0.5 - 2*(alpha**2))

def metropolis(N, alpha):
    L = 3/np.sqrt(2*alpha) 
    z = np.random.rand()*2*L-L 
    E = 0
    E_traj = np.zeros(N) 
    E2 = 0
    accepted = 0 
    rejected = 0 
    #Algorithm
    for i in range(N):
        z_trial = z + 0.9*(np.random.rand()*2*L-L) 
        if prob_density(alpha, z_trial) >= prob_density(alpha, z):
            z = z_trial
            accepted += 1
        else:
            dummy = np.random.rand()
            if dummy < prob_density(alpha, z_trial)/prob_density(alpha, z):
                z = z_trial
                accepted += 1
            else:
                rejected += 1
        E += E_local(alpha, z)/N
        E_traj[i] = E_local(alpha, z)
        E2 += E_local(alpha, z)**2/N
    
    acceptance_ratio = accepted/(accepted+rejected)
        
    
    return E, E_traj, E2, acceptance_ratio


def analytical_energy(alpha):
    return 0.5*alpha + 1/(8*alpha)

def analytical_variance(alpha):
    return (1-4*(alpha**2))**2/(32*(alpha**2))  

# Golden Section Search
def golden_section_search(f, a, b, tol=1e-3):
    gr = (np.sqrt(5) + 1) / 2
    c = b - (b - a) / gr
    d = a + (b - a) / gr
    while abs(b - a) > tol:
        if f(c) < f(d):
            b = d
        else:
            a = c
        c = b - (b - a) / gr
        d = a + (b - a) / gr
    return (b + a) / 2

def run_metropolis_and_report(alpha, N):
    E, E_traj, E2, acceptance_ratio = metropolis(N, alpha)
    E_mean = np.mean(E_traj)
    E2_mean = np.mean(E_traj**2)
    var_E = E2_mean - E_mean**2
    
    E_analytical = analytical_energy(alpha)
    var_analytical = analytical_variance(alpha)
    
    print('Alpha: %0.4f' % alpha, '<E>: %0.4f' % E_mean, 'E_analytical: %0.4f' % E_analytical,
          'VarE: %0.4f' % var_E, 'Var_analytical: %0.4f' % var_analytical,
          'acceptance ratio: %0.4f' % acceptance_ratio)
    
    return E_mean


## Mainn ##
N = 1000000 
alpha_min = golden_section_search(lambda alpha: run_metropolis_and_report(alpha, N), 0.1, 1.5)

print("\nOptimal alpha found:", alpha_min)

################# Analysis #############################

# Energy and variance 
alphas = np.linspace(0.1, 1.5, 20)  
E_means = []
variances = []

for alpha in alphas:
    E, E_traj, E2, acceptance_ratio = metropolis(N, alpha)
    E_mean = np.mean(E_traj)
    var_E = np.mean(E_traj**2) - E_mean**2
    E_means.append(E_mean)
    variances.append(var_E)

plt.figure(figsize=(8,5))
plt.plot(alphas, E_means, 'o-', label='Simulated <E>')
plt.plot(alphas, analytical_energy(alphas), 'r--', label='Analytical <E>')
plt.xlabel(r'$\alpha$')
plt.ylabel(r'$\langle E \rangle$')
plt.title('Mean local energy vs alpha')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(8,5))
plt.plot(alphas, variances, 'o-', label='Simulated Variance')
plt.plot(alphas, analytical_variance(alphas), 'r--', label='Analytical Variance')
plt.xlabel(r'$\alpha$')
plt.ylabel('Variance of $E$')
plt.title('Variance of local energy vs alpha')
plt.legend()
plt.grid(True)
plt.show()

# Histogram of visited points
alpha = alpha_min
E, E_traj, E2, acceptance_ratio = metropolis(N, alpha)


def metropolis_return_z(N, alpha):
    L = 3/np.sqrt(2*alpha)
    z = np.random.rand()*2*L-L
    z_points = []  
    for i in range(N):
        z_trial = z + 0.9*(np.random.rand()*2*L-L)
        if prob_density(alpha, z_trial) >= prob_density(alpha, z):
            z = z_trial
        else:
            dummy = np.random.rand()
            if dummy < prob_density(alpha, z_trial)/prob_density(alpha, z):
                z = z_trial
        z_points.append(z)
    return np.array(z_points)


z_points = metropolis_return_z(N, alpha)

plt.figure(figsize=(8,5))
z_grid = np.linspace(-3, 3, 500)
plt.hist(z_points, bins=100, density=True, alpha=alpha_min, label='Visited points histogram')
plt.plot(z_grid, prob_density(alpha, z_grid), 'r--', label='Trial wavefunction $|\psi(z)|^2$')
plt.xlabel('z')
plt.ylabel('Probability density')
plt.title('Histogram of visited points and trial wavefunction')
plt.legend()
plt.grid(True)
plt.show()

2D (VMC)

In [ ]:
@njit
def WaveFunction(r, lambda_param, alpha):
    r1_sq = r[0, 0]**2 + r[0, 1]**2
    r2_sq = r[1, 0]**2 + r[1, 1]**2

    dx = r[0, 0] - r[1, 0]
    dy = r[0, 1] - r[1, 1]
    r12 = np.sqrt(dx*dx + dy*dy)

    return np.exp(-0.5*(r1_sq + r2_sq)) * np.exp((lambda_param * r12) / (1 + alpha * r12))

@njit
def local_energy(r, lambda_param, alpha):
    # r: 2x2 matrix
    x1, y1 = r[0]
    x2, y2 = r[1]

    
    dx = x1 - x2
    dy = y1 - y2
    r12 = np.sqrt(dx*dx + dy*dy)
    
    if r12 < 1e-10:  
        r12 = 1e-10

    def derivative_factor(r, xi, xj, alpha, lambda_param):
        one_plus_alpha_r = 1 + alpha * r
        one_plus_alpha_r2 = one_plus_alpha_r**2
        one_plus_alpha_r3 = one_plus_alpha_r**3

        term1 = -1
        term2 = lambda_param / (one_plus_alpha_r2 * r)
        term3 = -lambda_param * (xi - xj)**2 * (1 + 3 * alpha * r) / (one_plus_alpha_r3 * r**3)
        term4 = (-xi + lambda_param * (xi - xj) / (one_plus_alpha_r2 * r))**2

        return term1 + term2 + term3 + term4

    # Second derivatives
    d2x1 = derivative_factor(r12, x1, x2, alpha, lambda_param)
    d2y1 = derivative_factor(r12, y1, y2, alpha, lambda_param)
    d2x2 = derivative_factor(r12, x2, x1, alpha, lambda_param)
    d2y2 = derivative_factor(r12, y2, y1, alpha, lambda_param)

    # Harmonic oscillator 
    potential = 0.5 * (x1**2 + y1**2 + x2**2 + y2**2)

    coulomb = lambda_param / r12

    kinetic = -0.5 * (d2x1 + d2y1 + d2x2 + d2y2)


    local_energy = kinetic + potential + coulomb

    return local_energy

@njit
def metropolis_2e(N, alpha, lambda_param, step_size=2.2):
    np.random.seed(42)
    r = np.random.randn(2, 2)  
    wf_old = WaveFunction(r, lambda_param, alpha)
    E_traj = np.zeros(N) 
    accepted = 0 
    rejected = 0 

    for i in range(N):
        r_trial = r + step_size * (np.random.rand(2, 2) - 0.5)
        wf_new = WaveFunction(r_trial, lambda_param, alpha)
        p_accept = (wf_new / wf_old)**2

        if p_accept >=1:
            r = r_trial
            wf_old = wf_new
            accepted += 1
        else:
            dummy = np.random.rand()
            if dummy < p_accept:
                r = r_trial
                wf_old = wf_new
                accepted += 1
            else:
                rejected += 1
        E_traj[i] = local_energy(r, lambda_param, alpha)
        

    E_mean = np.mean(E_traj)
    E2_mean = np.mean(E_traj**2)
    E_var = E2_mean - E_mean**2
    acceptance_ratio = accepted/(accepted+rejected)
    
    return E_mean, E_traj, E_var, acceptance_ratio


# Golden Section Search
def golden_section_search(f, a, b, tol=1e-3):
    gr = (np.sqrt(5) + 1) / 2
    c = b - (b - a) / gr
    d = a + (b - a) / gr
    while abs(b - a) > tol:
        if f(c) < f(d):
            b = d
        else:
            a = c
        c = b - (b - a) / gr
        d = a + (b - a) / gr
    return (b + a) / 2


def run_metropolis_and_report(alpha, N, lambda_param):
    E_mean, E_traj, E_var, acceptance_ratio = metropolis_2e(N, alpha, lambda_param)

    #print(f"Alpha: {alpha:.4f}, <E>: {E_mean:.4f}, Var[E]: {E_var:.4f}, Acceptance ratio: {acceptance_ratio:.4f}")

    return E_mean

### Main ####

lambda_param = 0.0
N = 2000000

alpha_min = golden_section_search(lambda alpha: run_metropolis_and_report(alpha, N, lambda_param), 0.1, 1.0)

print(f"\nOptimal alpha: {alpha_min:.4f}")

############ Analysis #####################

alpha_vals = np.linspace(0.1, 1.0, 30)
energies = []
variances = []

for alpha in alpha_vals:
    E_mean, _, E_var, _ = metropolis_2e(N, alpha, lambda_param)
    energies.append(E_mean)
    variances.append(E_var)
    #print(f"Alpha: {alpha:.4f}, Energy: {E_mean:.4f}, Variance: {E_var:.4f}")


fig, ax1 = plt.subplots(figsize=(10, 6))


color1 = 'tab:blue'
ax1.set_xlabel(r'$\alpha$')
ax1.set_ylabel('Mean Energy', color=color1)
ax1.plot(alpha_vals, energies, label='Mean Energy', color=color1, marker='o')
ax1.tick_params(axis='y', labelcolor=color1)
ax1.axvline(alpha_min, color='gray', linestyle='--', label=f'Optimal alpha = {alpha_min:.3f}')

ax2 = ax1.twinx()
color2 = 'tab:orange'
ax2.set_ylabel('Variance of Energy', color=color2)
ax2.plot(alpha_vals, variances, label='Variance', color=color2, marker='x')
ax2.tick_params(axis='y', labelcolor=color2)

plt.title('Energy and Variance vs. Alpha')
fig.tight_layout()
plt.grid(True)

lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper center')

plt.show()

# Only energy vs alpha
plt.figure(figsize=(10, 6))
plt.plot(alpha_vals, energies, label='Mean Energy', marker='o', color='blue')
plt.axvline(alpha_min, color='gray', linestyle='--', label=f'Optimal alpha = {alpha_min:.3f}')
plt.xlabel(r'$\alpha$')
plt.ylabel(r'$\langle E \rangle$')
plt.title('Energy vs. Alpha')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

########## Prob. Distribution #####################

r1_fixed = np.array([0.7, 0.0])

grid_points = 100
x = np.linspace(-5, 5, grid_points)
y = np.linspace(-5, 5, grid_points)
X, Y = np.meshgrid(x, y)
Z = np.zeros_like(X)

for i in range(grid_points):
    for j in range(grid_points):
        r2 = np.array([X[i, j], Y[i, j]])
        r = np.array([r1_fixed, r2])
        psi = WaveFunction(r, lambda_param, alpha_min)
        Z[i, j] = psi**2

# Normalize
Z /= np.sum(Z) * (x[1] - x[0]) * (y[1] - y[0])  

# 3D:
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z, cmap='plasma', edgecolor='none')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel(r'$|\Psi|^2$')
plt.tight_layout()
plt.show()

# 2D Heatmap:
plt.figure(figsize=(8, 6))
plt.contourf(X, Y, Z, levels=100, cmap='plasma')
plt.colorbar(label=r'$|\Psi|^2$')
plt.plot(r1_fixed[0], r1_fixed[1], 'ro', label='Fixed Particle 1')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.legend()
plt.axis('equal')
plt.tight_layout()
plt.show()

